<img src='https://hammondm.github.io/hltlogo1.png' style="float:right">

Linguistics 578<br>
Fall 2023<br>
Hammond

## Things to remember about any homework assignment:

1. For this assignment, you will edit this jupyter notebook and turn it in. Do not turn in pdf files or separate `.py` files.
1. Late work is not accepted.
1. Given the way I grade, you should try to answer *every* question, even if you don't like your answer or have to guess.
1. You may *not* use `python` modules that we have not already used in class.
1. You may certainly talk to your classmates about the assignment, but everybody must turn in *their own* work. It is not acceptable to turn in work that is essentially the same as the work of classmates.
1. All code must run. It doesn't have to be perfect, it may not do all that you want it to do, but it must run without error.
1. Code must run in reasonable time. Assume that if it takes more than *5 minutes* to run (on your machine), that's too long.
1. Please do not add, remove, or copy autograded cells.
1. Make sure to select `restart, run all cells` from the `kernel` menu when you're done and before you turn this in!

***

***my name***: Ashwin Raj

***people I talked to about the assignment***: [put your answer here]

***

## Homework #4

Here are the imports. Please do not import anything else.

In [1]:
import graphviz,re
import pywrapfst as fst
import numpy as np

1. Using `pywrapfst`, create a symbol table for the (lowercase) English alphabet plus $\epsilon$ and `#`. (The second symbol added should be `#`.)

In [2]:
st = fst.SymbolTable()
eng = "abcdefghijklmnopqrstuvwxyz"
letters = []
letters.append("<epsilon>")
letters.append("#")
for char in eng:
    letters.append(char)
for letter in letters:
    st.add_symbol(letter)

In [3]:
assert st.num_symbols() == 28

In [4]:
assert st.find(0) == '<epsilon>'

In [5]:
assert st.find(1) == '#'

2. We'll now build a character *bigram model* based on the *Alice in Wonderland* text. The following function should strip the Gutenberg information at the beginning and end of the file, convert everything to lowercase, convert everything except lowercase letters to space, and return a list of lowercase words.

In [6]:
def getwords(alice):
    '''tokenizes the Alice text
    args:
        alice: the location of the Gutenberg alice file
    returns
        a list of lowercase words
    '''
    # YOUR CODE HERE
    f = open(alice,'r')
    t = f.read()
    f.close()
    t = t[1435:]
    t = t[:-18590]
    t = t.lower()
    #print(t[:40])
    #t = t.strip()
    t = re.sub(r"[^a-z]+"," ",t)
    #t = t.replace("\n",'')
    #t = re.split(" ",t)
    words = t.split()
    #print(words[:12])
    return words

In [7]:
words = getwords('alice.txt')
assert words[:12] == [
    'chapter', 'i', 'down', 'the', 'rabbit', 'hole',
    'alice', 'was', 'beginning', 'to', 'get', 'very'
]

In [8]:
assert words[-5:] == ['happy', 'summer', 'days', 'the', 'end']

In [9]:
assert len(words) == 27336

3. We now write a function that will return a list of counts for letter unigrams and letter bigrams. You'll need to pad each word with `#` on each side before doing this.

In [10]:
def getcounts(wds):
    '''get letter unigram and bigram counts from a list of words
    args:
        words: a list of words
    returns
        unigrams: a dictionary from letters to counts
        bigrams: a dictionary from letter pairs to counts
    '''
    # YOUR CODE HERE
    unigrams = {}
    bigrams = {}
    new_words = ["#" + word + "#" for word in wds]
    for word in new_words:
        for char in word:
            if char not in unigrams:
                unigrams[char] = 1
            else:
                unigrams[char]+=1
    for word in new_words:
        for i in range(len(word)-1):
            bigram = word[i] + word[i+1]
            if bigram not in bigrams:
                bigrams[bigram] = 1
            else:
                bigrams[bigram]+=1
                
    return unigrams,bigrams
    


In [11]:
ugs,bgs = getcounts(words)
assert len(ugs) == 27

In [12]:
assert len(bgs) == 427

In [13]:
assert bgs['ab'] == 214

In [14]:
assert ugs['#'] == 54672

In [15]:
assert ugs['q'] == 209

4. Let's now write a function that takes our unigram and bigram counts and creates a dictionary of log conditional probabilities.

In [32]:
def makecondprobs(unigrams,bigrams):
    '''calculate conditional probabilities
    args:
        unigrams: dictionary of unigram counts
        bigrams: dictionary of bigram counts
    returns:
        a dictionary of log conditional probabilities
    '''
    # YOUR CODE HERE
    conditional_probs = {}
    for bigram, bigram_count in bigrams.items():
        unigram1, unigram2 = bigram[0], bigram[1] 
        if unigram1 in unigrams:
            unigram1_count = unigrams[unigram1]
        else:
            unigram1_count = 0
        numerator = bigram_count
        denominator = unigram1_count
        prob = (numerator) / (denominator)
        log_prob = np.log(prob)
        conditional_probs[bigram] = log_prob

    return conditional_probs
print(st)
print(cp)

<SymbolTable '<unspecified>' at 0x7fd9d4544fd0>
{'#c': -4.093466984280742, 'ch': -1.6723083764709745, 'ha': -1.8609460640786497, 'ap': -4.389794551275357, 'pt': -3.3217605510141204, 'te': -2.655377676760762, 'er': -2.008549362353763, 'r#': -1.2669384038976261, '#i': -3.313719695501101, 'i#': -2.6083663961840644, '#d': -4.148692283271645, 'do': -2.4340599742998528, 'ow': -2.7150755581539268, 'wn': -2.956856824531197, 'n#': -1.237825166400289, '#t': -2.457626326304217, 'th': -1.1225706336106682, 'he': -0.6686364778585315, 'e#': -0.8414935018554685, '#r': -4.6287111353948776, 'ra': -3.0133661411550605, 'ab': -3.7151664184826494, 'bb': -3.1074362631339922, 'bi': -2.5966106393680017, 'it': -1.7345566592860768, 't#': -1.1396799388855952, '#h': -3.5369889460172863, 'ho': -2.5584619715520103, 'ol': -3.8988456551623436, 'le': -1.8655571182348962, '#a': -2.7775762637508206, 'al': -2.2481107007183003, 'li': -1.7072996564549847, 'ic': -2.5318072843137975, 'ce': -1.2247776030772974, '#w': -3.372743

In [33]:
cp = makecondprobs(ugs,bgs)
assert len(cp) == 427

In [18]:
assert np.isclose(cp['ab'],-3.71,atol=.01)

In [19]:
total = 0
for bigram in cp:
    if bigram[0] == 'a':
        total += np.exp(cp[bigram])
assert np.isclose(total,1.0,atol=.0001)

5. We now create a WFSA for this model using the symbol table you created in question #1 and the dictionary we just created. The state geometry is critical here. There will be a start state, which you can think of as the `#` on the left. From there, the first arc takes you to various letter states. The probabilities of those arcs will be the conditional probabilities of the different letters based on the initial `#`. Now from each letter state you can go to any other letter state or the final `#`. In each case, the relevant probability is the conditional probability of the second letter/`#` based on the first letter. The second `#` state is the only final state. Note that there will be no arc corresponding to the first `#`.

In [44]:
def makeWFSA(tab,dic):
    '''create a WFSA that encodes a bigram model
    args:
        tab: a symbol table including all letters
            and boundary markers
        dic: a dictionary of the conditional probabilities
    returns:
        a WFSA
    '''
    # YOUR CODE HERE
    
    c1 = fst.Compiler(isymbols=tab, osymbols=tab, arc_type="log",keep_isymbols=True,keep_osymbols=True)
    #boundary_id = tab.find("#")
    
    
    # Define symbol IDs for boundary markers and letters
    start_id = tab.find("<epsilon>")
    boundary_id = tab.find("#")

    # Set the start state
    #c1.write(f"0 {start_id}\n")
    c1.write(f"0 {boundary_id}\n")
    #c1.write(f"0 {start_id}\n")  # Skip state '1' as it's being used as a state identifier

    num_symbols = tab.num_symbols()

    # Add arcs from the start state to letter states with conditional probabilities
    state_index = 3  # Start at index 3 to skip state '1'
    for i in range(2, num_symbols):  # Start at index 2, skipping <epsilon> and #
        letter_id = i
        letter = tab.find(letter_id)
        prob = dic.get("#"+letter,0.0)
        if prob > 0.0:
            c1.write(f"0 {state_index} {letter_id} {letter_id} {prob}\n")
            state_index += 1

    # Add arcs from letter states to other letter states or the final state
    for i in range(2, num_symbols):  # Start at index 2, skipping <epsilon> and #
        letter_id=i
        letter = tab.find(letter_id)
        for j in range(2, num_symbols):  # Start at index 2, skipping <epsilon> and #
            next_letter_id = j
            next_letter = tab.find(next_letter_id)
            next_prob = dic.get(letter+next_letter,0.0)
            if next_prob > 0.0:
                c1.write(f"{i} {i} {letter_id} {next_letter_id} {next_prob}\n")
                #state_index += 1

        # Add an arc to the final state
        c1.write(f"{i} 1 {boundary_id} {boundary_id} 0\n")
        

    # Compile the WFSA and create an FST object
    #compiler.close()
    #w = fst.Fst.read_text(compiler.stdstr())
    w = c1.compile()

    return w
    
    


In [45]:
wfsa = makeWFSA(st,cp)
assert type(wfsa) == fst.MutableFst

LEVEL(FST_FLAGS_fst_error_fatal ? base_logging::FATAL : base_logging::ERROR): FstCompiler: Symbol "1" is not mapped to any integer arc ilabel, symbol table = <unspecified>, source = <pywrapfst>, line = 2
LEVEL(FST_FLAGS_fst_error_fatal ? base_logging::FATAL : base_logging::ERROR): FstCompiler: Symbol "1" is not mapped to any integer arc olabel, symbol table = <unspecified>, source = <pywrapfst>, line = 2
LEVEL(FST_FLAGS_fst_error_fatal ? base_logging::FATAL : base_logging::ERROR): FstCompiler: Symbol "1" is not mapped to any integer arc ilabel, symbol table = <unspecified>, source = <pywrapfst>, line = 3
LEVEL(FST_FLAGS_fst_error_fatal ? base_logging::FATAL : base_logging::ERROR): FstCompiler: Symbol "1" is not mapped to any integer arc olabel, symbol table = <unspecified>, source = <pywrapfst>, line = 3
LEVEL(FST_FLAGS_fst_error_fatal ? base_logging::FATAL : base_logging::ERROR): FstCompiler: Symbol "1" is not mapped to any integer arc ilabel, symbol table = <unspecified>, source = <p

FstOpError: Operation failed

In [ ]:
assert wfsa.num_states() == 28

In [ ]:
c2 = fst.Compiler(
    isymbols=st,
    osymbols=st,
    keep_isymbols=True,
    keep_osymbols=True,
    arc_type='log'
)
c2.write('0 1 h h')
c2.write('1 2 a a')
c2.write('2 3 t t')
c2.write('3 4 # #')
c2.write('4')
f2 = c2.compile()
f3 = fst.compose(f2,wfsa)
assert f3.num_states() == 5

In [ ]:
f3s = str(f3).split('\n')[:-2]
val = np.exp(sum([float(s.split('\t')[-1]) for s in f3s]))
assert np.isclose(val,0.00019,atol=.001)